In [2]:
import os, sys
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import json

import warnings
warnings.simplefilter("ignore", category=(PendingDeprecationWarning, DeprecationWarning))

# Import wavenetclass modules
from train import Train


In [4]:
tf.reset_default_graph()  # Clears the defined variables and operations of the previous cell
    
# Run
config = 'default.ini'
modes = ['train', 'val']
train = Train(config_file=config, modes=modes)
train.run()